**1.数据预处理**

In [4]:
import pandas as pd 
import numpy as np 

# create feature list
column_names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 
'Uniformity of Cell Shape', 'Marginal Adhesion','Single Epithelical Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']

# read data
data = pd.read_csv('/Users/scarlett/repository/projects/breast_cancer/breast_cancer.csv',names=column_names)

In [6]:
# clean data
data = data.replace(to_replace='?', value=np.nan)
data = data.dropna(how='any')
print data.shape
print data.info()

(683, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
Sample code number              683 non-null int64
Clump Thickness                 683 non-null int64
Uniformity of Cell Size         683 non-null int64
Uniformity of Cell Shape        683 non-null int64
Marginal Adhesion               683 non-null int64
Single Epithelical Cell Size    683 non-null int64
Bare Nuclei                     683 non-null object
Bland Chromatin                 683 non-null int64
Normal Nucleoli                 683 non-null int64
Mitoses                         683 non-null int64
Class                           683 non-null int64
dtypes: int64(10), object(1)
memory usage: 64.0+ KB
None


由于原始数据没有提供对应的测试样本，故需要对带有label的样本进行分割，一般是25%作为测试集，75%作为训练集。

**2.准备训练、测试数据**

In [10]:
# prepare training set and testing set

# use train_test_split in sklearn to split data
from sklearn.cross_validation import train_test_split

# randomly sample 25% for testing,75% for training
X_train,X_test,y_train,y_test = train_test_split(data[column_names[1:10]],data[column_names[10]],test_size=0.25,random_state=33)

# check number and class
print y_train.value_counts()
print y_test.value_counts()

2    344
4    168
Name: Class, dtype: int64
2    100
4     71
Name: Class, dtype: int64


训练样本：512条（344条良性肿瘤数据+168恶性肿瘤数据），测试样本171条（100+71）

**sklearn.model_selection.train_test_split解释**

from sklearn.cross_validation import train_test_split

一般形式：X_train,X_test, y_train, y_test = cross_validation.train_test_split(train_data,train_target,test_size=0.4, random_state=0)

参数解释：

- train_data：所要划分的样本特征集
- train_target：所要划分的样本结果
- test_size：样本占比，如果是整数的话就是样本的数量
- random_state：是随机数的种子

> 随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。

> 随机数的产生取决于种子，随机数和种子之间的关系遵从以下两个规则：

> 种子不同，产生不同的随机数；种子相同，即使实例不同也产生相同的随机数。

**3.使用线性分类模型进行分类预测**

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# standarlize data,make sure DE=1,EX=0,so that the outcome wont be influenced by big feature
ss = StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

# init logisticregression and SGDClassifier
lr = LogisticRegression()
sgdc = SGDClassifier()

# use fit() of LR to train paras
lr.fit(X_train,y_train)
# use trained lr to predict X_test
lr_y_predict = lr.predict(X_test)

# use fit() of SGDC to train paras, use trained lr to predict X_test
sgdc.fit(X_train,y_train)
sgdc_y_predict=sgdc.predict(X_test)

print sgdc_y_predict
print lr_y_predict

[4 2 4 4 2 2 2 4 2 2 2 2 4 2 4 4 4 4 4 2 2 4 4 2 4 4 2 2 4 4 4 4 4 4 4 4 2
 4 4 4 4 4 2 4 2 2 4 2 2 4 4 2 2 2 4 2 2 2 2 2 4 4 2 2 2 4 2 2 2 2 4 2 2 4
 2 2 2 2 4 2 2 2 4 2 2 2 4 2 4 2 4 4 2 2 2 2 4 4 2 2 2 4 2 2 4 2 2 2 2 2 4
 2 2 2 2 2 2 4 2 2 4 4 2 4 2 2 2 4 2 2 4 4 2 4 4 2 2 2 2 4 2 4 2 4 2 2 2 2
 2 4 4 2 4 4 2 4 2 2 2 2 4 4 4 2 4 2 2 4 2 4 4]
[2 2 4 4 2 2 2 4 2 2 2 2 4 2 4 4 4 4 4 2 2 4 4 2 4 4 2 2 4 4 4 4 4 4 4 4 2
 4 4 4 4 4 2 4 2 2 4 2 2 4 4 2 2 2 4 2 2 2 2 2 4 4 2 2 2 4 2 2 2 2 4 2 2 2
 2 2 2 4 4 2 2 2 4 2 2 2 4 2 4 2 4 4 2 2 2 2 4 4 2 2 2 4 2 2 4 2 2 2 2 2 4
 2 2 2 2 2 2 4 2 2 4 4 2 4 2 2 2 4 2 2 4 4 2 4 4 2 2 2 2 4 2 4 2 4 2 2 2 2
 2 4 4 2 4 4 2 4 2 2 2 2 4 4 4 2 4 2 2 4 2 4 4]


混淆矩阵：
二分类任务中，预测结果(predicted condition)与正确标记(true condition)之间存在4种不同的组合：

- 真阳性(true positive)：预测正确的恶性肿瘤
- 真阴性
- 假阳性(false positive)：误判为恶性肿瘤
- 假阴性

**4.性能评价**

**性能评价指标**

评价指标1：准确率

$ Accuracy = \frac{TP + TN}{TP+TN+FP+FN}$

评价指标2：召回率(Recall)和精确率(Precision),F1 指标(F1 measure)

$$ Precision = \frac{TP}{TP+FP} $$
$$ Recall = \frac{TP}{TP+FN}$$
$$ F1 measure = \frac{2}{\frac{1}{Precision}+\frac{1}{Recall}}$$ 

F1指标：两指标的调和平均数，以综合考量两指标

对肿瘤识别，我们更关心召回率，即应该被正确识别的恶性肿瘤的百分比。

**使用线性分类模型进行肿瘤预测任务的性能分析**

In [20]:
from sklearn.metrics import classification_report

# 使用逻辑回归模型自带的评分函数score获得模型在测试集上的准确性结果
print 'Acurracy of LR Classifier:', lr.score(X_test,y_test)
# use classification_report to get the other 3 measures of LR
print classification_report(y_test,lr_y_predict,target_names=['Benign','Malignant'])

# 使用随机梯度下降模型自带的score评分函数模型在测试集上的准确性结果
print 'Acurracy of SDG Classifier:', sgdc.score(X_test,y_test)
# use classification_report to get the other 3 measures of SGDC
print classification_report(y_test,lr_y_predict,target_names=['Benign','Malignant'])

Acurracy of SDG Classifier: 0.9824561403508771
             precision    recall  f1-score   support

     Benign       0.99      0.99      0.99       100
  Malignant       0.99      0.99      0.99        71

avg / total       0.99      0.99      0.99       171



综上，发现，LR比SGDC在测试集上有更高的准确性，因为sklearn中采用解析的方式精确计算LR的参数，而使用梯度法估计SGDC的参数

特点分析：

- LR model：精确解析参数，计算时间长但模型性能略高
- SGDC model：随机梯度上升算法估计参数，计算时间短但模型性能略低
- 训练数据规模在10万量级以上的数据，考虑到时间耗用，推荐使用随机梯度算法对模型参数进行估计